In [1]:
#%load_ext autoreload
#%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.models.boxatlas_parametric import BoxAtlas, MovingLimb, FixedLimb, Trajectory
from pympc.dynamical_systems import dare, moas_closed_loop
from pympc.control import MPCHybridController, FeasibleSetLibrary, load_library
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt
import director.viewerclient as vc
from director.thirdparty import transformations

# Construction of Box-Atlas

Numeric parameters

In [3]:
parameters = {
    'mass': 1.,
    'stiffness': 200.,
    'damping': 100.,
    'friction_coefficient': .5,
    'gravity': 10.,
    'sampling_time': .1,
    'integrator': 'explicit_euler'
    }

Nominal limb position and force

In [4]:
weight = parameters['mass'] * parameters['gravity']
penetration = weight / parameters['stiffness']
nominal_configuration = {
    'b': np.zeros((2,1)),
    'lf': np.array([[.2], [-.5 - penetration / 2.]]),
    'rf': np.array([[-.2], [-.5 - penetration / 2.]]),
    'lh': np.array([[.3], [.0]]),
    'rh': np.array([[-.3], [.0]])
    }
nominal_contact_forces = {
    'lf': np.array([[weight / 2.], [0.]]),
    'rf': np.array([[weight / 2.], [0.]]),
    'lh': np.zeros((2,1)),
    'rh': np.zeros((2,1))
    }

State and input limits

In [5]:
kinematic_limits = {
        'b': {'min': np.array([[-.2],[-.1]]), 'max': np.array([[.2],[.1]])},
        'lf': {'min': np.array([[.0],[-.7]]), 'max': np.array([[.4],[-.3]])}, # wrt body
        'rf': {'min': np.array([[-.4],[-.7]]), 'max': np.array([[0.],[-.3]])}, # wrt body
        'lh': {'min': np.array([[.2],[-.2]]), 'max': np.array([[.4],[.2]])}, # wrt body
        'rh': {'min': np.array([[-.4],[-.2]]), 'max': np.array([[-.2],[.2]])}, # wrt body
        }
velocity_limits = {
        'b': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'lf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'rf': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'lh': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        'rh': {'min': -np.ones((2,1)), 'max': np.ones((2,1))},
        }
f_min = np.array([[0.], [-parameters['friction_coefficient'] * weight]])
f_max = np.array([[weight], [parameters['friction_coefficient'] * weight]])
force_limits = {
        'lf': {'min': f_min*2., 'max': f_max*2.},
        'rf': {'min': f_min*2., 'max': f_max*2.},
        'lh': {'min': f_min, 'max': f_max},
        'rh': {'min': f_min, 'max': f_max},
        }

Robot moving limbs

In [6]:
# left hand
A_domains_lh = [
    np.array([[1., 0.]]), # not in contact
    np.array([[-1., 0.],[1., 0.]]) # in contact
]
b_domains_lh = [
    np.array([[.5]]), # not in contact
    np.array([[-.5],[.5+penetration]]) # in contact
]
contact_surfaces_lh = [None, 0]
lh = MovingLimb(
    A_domains_lh,
    b_domains_lh,
    contact_surfaces_lh
)

# right hand
A_domains_rh = [
    #-np.eye(2), # center
    #np.array([[1., 0.],[0., -1.]]), # over the table
    np.array([[-1., 0.],[-1., 1.]]), # center
    np.array([[0., -1.],[1., -1.]]), # over the table
    np.array([[0., 1.],[0., -1.],[1., -1.]]), # top of the table
    np.array([[1., 0.],[-1., 0.],[-1., 1.]]) # side of the table
]
b_domains_rh = [
    #np.array([[.3],[.5]]), # center
    #np.array([[-.35],[0.]]), # over the table
    np.array([[.4],[.4]]), # center
    np.array([[0.],[-.4]]), # over the table
    np.array([[0.],[penetration],[-.4]]), # top of the table
    np.array([[-.4],[.4 + penetration],[.4]]) # side of the table
]
contact_surfaces_rh = [None, None, 0, 0]
forbidden_transitions_rh = [(2,3),(3,2),(0,2),(2,0),(1,3),(3,1)]
rh = MovingLimb(
    A_domains_rh,
    b_domains_rh,
    contact_surfaces_rh,
    forbidden_transitions_rh
)

Robot fixed limbs

In [7]:
lf = FixedLimb(np.array([[.2],[-.5]]), np.array([[0.],[1.]]))
rf = FixedLimb(np.array([[-.2],[-.5]]), np.array([[0.],[1.]]))

Assemble robot

In [8]:
limbs = { 'moving': {'lh': lh, 'rh': rh}, 'fixed': {'lf': lf, 'rf': rf} }
box_atlas = BoxAtlas(
    limbs,
    parameters,
    nominal_configuration,
    nominal_contact_forces,
    kinematic_limits,
    velocity_limits,
    force_limits
)

Empty polytope!
Empty polytope!


Print state, input, and modes (a mode indicates in which domain each moving limb is, domains are numbered as they are listed in the limb list)

In [9]:
box_atlas.print_state()
box_atlas.print_input()
print 'Box-atlas modes:\n', box_atlas.contact_modes

Box-Atlas states:
['qrhx', 'qrhy', 'qlhx', 'qlhy', 'qbx', 'qby', 'vbx', 'vby']
Box-Atlas inputs:
['vrhx', 'vrhy', 'vlhx', 'vlhy', 'flfn', 'flft', 'frfn', 'frft']
Box-atlas modes:
[{'rh': 0, 'lh': 0}, {'rh': 0, 'lh': 1}, {'rh': 1, 'lh': 0}, {'rh': 1, 'lh': 1}, {'rh': 2, 'lh': 0}, {'rh': 3, 'lh': 0}]


# Hybrid MPC Controller

Initial states for the Humanoids paper

In [10]:
# left hand
x_0_lh = np.array([
    [0.], # q_rh_x
    [0.], # q_rh_y
    [0.], # q_lh_x
    [0.], # q_lh_y
    [0.], # q_b_x
    [0.], # q_b_y
    [1.], # v_b_x
    [-.5] # v_b_y
])

# right hand top
x_0_rht = np.array([
    [-.1], # q_rh_x
    [.15], # q_rh_y
    [.0], # q_lh_x
    [.15], # q_lh_y
    [-.05], # q_b_x
    [.1], # q_b_y
    [-1.], # v_b_x
    [-.5] # v_b_y
])

# right hand side
x_0_rhs = np.array([
    [.1], # q_rh_x
    [-.21], # q_rh_y
    [.1], # q_lh_x
    [-.21], # q_lh_y
    [.1], # q_b_x
    [-.05], # q_b_y
    [-1.], # v_b_x
    [-.5] # v_b_y
])

Hybrid MPC controller

In [11]:
N = 10
Q = box_atlas.penalize_relative_positions(np.eye(box_atlas.n_x))
R = np.eye(box_atlas.n_u)
objective_norm = 'two'
terminal_mode = 0
P, K = dare(
    box_atlas.pwa_system.affine_systems[terminal_mode].A,
    box_atlas.pwa_system.affine_systems[terminal_mode].B,
    Q,
    R)
X_N = moas_closed_loop(
    box_atlas.pwa_system.affine_systems[terminal_mode].A,
    box_atlas.pwa_system.affine_systems[terminal_mode].B,
    K,
    box_atlas.pwa_system.domains[terminal_mode])
controller = MPCHybridController(box_atlas.pwa_system, N, objective_norm, Q, R, P, X_N)

Computation of MOAS started... MOAS found.
MOAS facets are 360, removing redundant ones... redundant factes removed, MOAS facets are 125.


Trick to avoid forbidden transitions

In [12]:
controller = box_atlas.avoid_forbidden_transitions(controller)

Closed-loop system simulation of the MPC controller

In [ ]:
N_sim = 30
u = []
x_0 = np.array([
    [.0], # q_rh_x
    [.0], # q_rh_y
    [.0], # q_lh_x
    [.0], # q_lh_y
    [.0], # q_b_x
    [.0], # q_b_y
    [-1.], # v_b_x
    [-.3] # v_b_y
])
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
for k in range(N_sim):
    u_k, x_k, ss_k = controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print('Time step ' + str(k) + ': ' + str(ss_k) + '.')
    u_ws = u_k[1:] + [K.dot(x_k[-1])]
    x_ws = x_k[1:] + [box_atlas.pwa_system.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (terminal_mode,)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k[0]])[0][1]
    #print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)
traj_miqp = Trajectory(x, u, Q, R, P)
#np.save('box_atlas_tajectory_miqp_lh', traj_miqp)

Plot of the state and the input

In [ ]:
mpc_plt.input_sequence(u, parameters['sampling_time'])
plt.show()
mpc_plt.state_trajectory(x, parameters['sampling_time'])
plt.show()

# Control with Approximate Hybrid MPC

Initialization of the library

In [ ]:
#library = FeasibleSetLibrary(controller)
samples_so_far = 100
library = load_library('library_box_atlas_wall_and_table_' + str(samples_so_far))

Coverage

In [ ]:
sample_step = 100
n_steps = 9
for i in range(n_steps):
    n = samples_so_far + (i+1)*sample_step
    library.sample_policy(sample_step, box_atlas.is_inside_a_domain)
    print(str(n) + ' samples added to the library.')
    library.save('library_box_atlas_wall_and_table_' + str(n))

Sample 0: 
solving MIQP...  solution found in 17.3733460903 s.
included.
including sample in inner approximation...  sample included in 10.209939003 s.
Sample 1: 
solving MIQP...  solution found in 0.00919795036316 s.
unfeasible.
Sample 2: 
solving MIQP...  solution found in 3.04666590691 s.
included.
including sample in inner approximation...  sample included in 8.39729595184 s.
Sample 3: 
solving MIQP...  solution found in 179.029603958 s.
new switching sequence (2, 2, 4, 5, 5, 0, 0, 0, 0, 0).
condensing QP...  QP condensed in 0.00572896003723 s.
constructing inner simplex...  inner simplex constructed in 0.000433206558228 s.
including sample in inner approximation...  sample included in 19.7787189484 s.
Sample 4: 
solving MIQP...  solution found in 0.0581798553467 s.
unfeasible.
Sample 5: 
solving MIQP...  solution found in 0.00499510765076 s.
unfeasible.
Sample 6: 
solving MIQP...  solution found in 2.70685505867 s.
included.
including sample in inner approximation...  sample inclu

Shifted switching sequences

In [ ]:
library.add_shifted_switching_sequences(terminal_mode)

Closed-loop system simulation of the approximate MPC controller

In [ ]:
N_sim = 50
u = []
x = [x_0_lh*0.]
ss = []
ss_feasible = None
for k in range(N_sim):
    print('Time step ' + str(k) + '.')
    u_k, ss_k = library.feedback(x[k], ss_feasible, max_qp=1)
    u.append(u_k)
    x_next = box_atlas.pwa_system.simulate(x[k], [u_k])[0][1]
    x.append(x_next)
    ss.append(ss_k)
    ss_feasible = ss_k[1:] + (terminal_mode,)
traj_lib = Trajectory(x, u, Q, R, P)
#np.save('box_atlas_tajectory_lib_lh', traj)

# Test Coverage

Number of feasible sets in the library

In [ ]:
n_fs = 0
for ss in library.library.values():
    if not ss['feasible_set'].empty:
        n_fs += 1
print 'Number of mode sequences in the library:', n_fs

Check coverage of the feasible set with n_samples

In [ ]:
n_samples = 20
cost_miqp = []
cost_qp = []
for i in range(n_samples):
    print i
    x_sample = library.random_sample(box_atlas.is_inside_a_domain)
    cost_miqp.append(controller.feedforward(x_sample)[3])
    cost_qp.append(library.feedforward(x_sample)[3])
#np.save('cost_random_miqp', cost_miqp)
#np.save('cost_random_qp', cost_qp)

Cost error as a function of the maximum number of QPs online

In [ ]:
feasible_samples = 0
samples_not_covered = 0
optimality_loss = []
for i, c_mi in enumerate(cost_miqp):
    if not np.isnan(c_mi):
        feasible_samples += 1
        if len(cost_qp[i]) == 0:
            samples_not_covered += 1
        else:
            optimality_loss.append([(c - c_mi)/c_mi for c in cost_qp[i]])
max_overlapping_qp = max([len(ol) for ol in optimality_loss])
loss = []
for pos in range(max_overlapping_qp):
    average_loss = 0
    for ol in optimality_loss:
        length = min(len(ol),pos+1)
        average_loss += min(ol[:length])
    loss.append(average_loss/float(len(optimality_loss))*100)
print 'Optimality loss (%) as a function of the maximum number of QPs solved online:\n', loss

## Visualizer

In [ ]:
box_atlas.visualize(x[0])
time.sleep(10 * parameters['sampling_time'])
for k in range(N_sim):
    box_atlas.visualize(x[k])
    time.sleep(parameters['sampling_time'])

In [ ]:
controller.__dict__.keys()